In [1]:
import orion_recommend
from orion_recommend.models import HybridMatrixFactorizationModel as HMFModel
import pandas as pd
import numpy as np
from orion_recommend.datasets import DatasetMap, synthetic, utils, Encoder
from orion.sources import S3Source
from orion.sources.io import read_csv, write_csv
from orion.sources import RedshiftSource
import pandas as pd
import numpy as np
from scipy import sparse
from orion_recommend.evaluate import metrics
import implicit
import lightfm

In [2]:
#source = RedshiftSource(query='SELECT * FROM publish.inventory_lookup WHERE productid IS NULL')
#df_stock = source.read_csv()

with S3Source(key="masters/uploads/customers/1560425511130_Peak_customers.csv", bucket="kilimanjaro-prod-datalake") as s3:
    df_cust = read_csv(s3)

df_trans = read_csv(S3Source(key="masters/uploads/transactions/1560426066385_Peak_transactions.csv", bucket="kilimanjaro-prod-datalake"))
df_prod = read_csv(S3Source(key="masters/uploads/product/1560425499995_Peak_product.csv", bucket="kilimanjaro-prod-datalake"))


/home/declanbarrycbc1/thesis/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (0,11) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
# Fix item number 
df_prod.itemnumber = pd.to_numeric(df_prod.itemnumber, errors='coerce') 

In [4]:
# Drop nas
df_prod.dropna(subset=["itemnumber"], inplace=True)

In [5]:
# Remove keys not in customer and product tables
df_trans = df_trans.iloc[df_trans.customerkey[df_trans.customerkey.isin(df_cust.customerkey)].index,:]
df_trans.reset_index(inplace=True, drop=True)
df_trans = df_trans.iloc[df_trans.itemnumber[df_trans.itemnumber.isin(df_prod.itemnumber)].index,:]

# Reset index before changing keys to integers
df_trans.reset_index(inplace=True, drop=True)
df_prod.reset_index(inplace=True, drop=True)


In [6]:
df_prod.dropna(inplace=True)

In [7]:
print(df_cust.customerkey.unique().shape)
df_cust.dropna(inplace=True)
print(df_cust.customerkey.unique().shape)

(2434521,)
(1701289,)


In [8]:
# Drop redundant cols. Fabric all unknown, item colour less info than colourvalue
df_prod.drop(["itemstylecode", "itemquarter", "itemcolour", "fabric"], axis=1, inplace=True)

In [9]:
# Drop columns
df_trans.drop(["salestransactionkey", "salesordernumber","discountpercent", "grosssales",
              'orderdate', 'ordertime', 'shippingdate','grossprofit', 'pricetype'], axis=1, inplace=True)

In [10]:
df_cust = df_cust.merge(df_trans, on = 'customerkey', how='left')

df_cust.drop(['itemnumber','unitssold'],axis=1, inplace=True)

In [11]:
df_trans.drop(['postcode', 'shipcountry', 'unitssold'], axis=1, inplace=True)

In [12]:
# Rename and drop columns, nas
df_trans.rename(columns={"customerkey":"user_id","itemnumber":"item_id" }, inplace=True)
df_trans.dropna(inplace=True)
df_trans.reset_index(inplace=True, drop=True)

In [13]:
df_cust.rename(columns={"customerkey":"user_id", "shipcountry":'country'}, inplace=True)
df_prod.rename(columns={"itemnumber":"item_id", "brandcode":"brand_id"}, inplace=True)

In [14]:
# Interactions
cats = df_trans.select_dtypes('object').columns
keys = df_trans[cats].apply(lambda x: x.factorize()[1])
df_trans[cats] = df_trans[cats].apply(lambda x: x.factorize()[0])

# Users
cats = df_cust.select_dtypes('object').columns
keys = df_cust[cats].apply(lambda x: x.factorize()[1])
df_cust[cats] = df_cust[cats].apply(lambda x: x.factorize()[0])
# Items
cats = df_prod.select_dtypes('object').columns
keys = df_prod[cats].apply(lambda x: x.factorize()[1])
df_prod[cats] = df_prod[cats].apply(lambda x: x.factorize()[0])

In [15]:
df_new_users = df_cust[~df_cust.user_id.isin(df_trans.user_id)]

In [16]:
np.save("new_users",df_new_users.to_numpy())

In [17]:
df_cust= df_cust[df_cust.user_id.isin(df_trans.user_id)]
df_trans= df_trans[df_trans.user_id.isin(df_cust.user_id)]


In [18]:
df_new_items = df_prod[~df_prod.item_id.isin(df_trans.item_id)]

In [19]:
np.save("new_items",df_new_items.to_numpy())

In [20]:
df_prod = df_prod[df_prod.item_id.isin(df_trans.item_id)]
df_trans = df_trans[df_trans.item_id.isin(df_prod.item_id)]

In [21]:
df_trans.user_id = df_trans.user_id.astype('int32')

In [22]:
df_cust.user_id = df_cust.user_id.astype('int32')

In [23]:
df_prod.item_id = df_prod.item_id.astype('int32')

In [24]:
df_trans.rename(columns={"user_id":"user", "item_id":'item'}, inplace=True)
df_cust.rename(columns={"customerkey":"user", "shipcountry":'country'}, inplace=True)
df_prod.rename(columns={"item_id":"item", "brandcode":"brand_id"}, inplace=True)

In [25]:
item_num = df_prod.item.unique().shape[0]

In [26]:
item_num

36126

In [27]:
#Renumber item ids
ints = [i for i in range(item_num)]
item_keys = [i for i in df_prod.item.unique()]
item_key_dict = {i:j for i,j in zip( item_keys, ints)}

# Change keys to integers in dataframes
df_prod.item = [item_key_dict[df_prod.item.iloc[i]] for i in range(len(df_prod))]

In [28]:
df_trans.item = [item_key_dict[df_trans.item.iloc[i]] for i in range(len(df_trans))]

### Final dataframes to be used


In [40]:
df_prod.to_csv("items")

df_cust.to_csv("users")

df_trans.to_csv("interactions")

### Orion baselines

In [29]:
train, test = DatasetMap.split(df_trans, users=df_cust, items=df_prod, frac_train=0.8)

/home/declanbarrycbc1/thesis/src/orion-recommend/orion_recommend/evaluate/utils.py:176: UserWarning: The total number of users that can be sampled from for your test set is less than 75% of the available users. Consider reducing yourchosen 'n_test' (it is set to 1).
  warnings.warn(
/home/declanbarrycbc1/thesis/src/orion-recommend/orion_recommend/datasets/dataset_map.py:222: UserWarning: There are elements in 'customer_list' that are not in 'users'.
  warnings.warn(
/home/declanbarrycbc1/thesis/src/orion-recommend/orion_recommend/datasets/dataset_map.py:222: UserWarning: There are elements in 'inventory' that are not in 'items'.
  warnings.warn(
/home/declanbarrycbc1/thesis/src/orion-recommend/orion_recommend/datasets/dataset_map.py:222: UserWarning: There are elements in 'customer_list' that are not in 'users'.
  warnings.warn(
/home/declanbarrycbc1/thesis/src/orion-recommend/orion_recommend/datasets/dataset_map.py:222: UserWarning: There are elements in 'inventory' that are not in 'i

In [30]:
model = HMFModel()

In [50]:
model.fit(train)

In [45]:
%%capture
preds = model.predict(test.items)


In [52]:
print(model.score(test, k=10, fn=metrics.truncated_ndcg))


TypeError: Invalid type passed to user_ids parameter. This must be either int or np.int32 array. Type received: <class 'numpy.int64'>

In [ ]:
fts = df_prod.iloc[:,1:]

In [ ]:
model.score(test,train, k=10, fn=metrics.ndcg)
model.score(test, train, k=10, fn=metrics.precision_at_k)

### Implicit

In [ ]:
from implicit.bpr import BayesianPersonalizedRanking as BPR


In [ ]:
mod = BPR()

In [ ]:
mod.fit(train)